In [1]:
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
import seaborn as sns
import pandas as pd
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
progress = ProgressBar().register()

In [4]:
IMG_DIR = '../../analysis'
DPI=120

## Loading data using Dask (loads lazily)
* https://www.youtube.com/watch?v=RA_2qdipVng&t=1s
* http://matthewrocklin.com/slides/scipy-2017.html

In [5]:
# data_types = {'CRSElapsedTime': int, 'CRSDepTime': int, 'Year': int, 'Month': int, 'DayOfWeek': int, 'DayofMonth': int}
data_types = {'CRSDepTime': int, 'Year': int, 'Month': int, 'DayOfWeek': int, 'DayofMonth': int}


# http://dask.pydata.org/en/latest/dataframe-overview.html
# %time df = dd.read_csv('../../data/raw/*.csv', encoding='iso-8859-1', dtype=data_types, assume_missing=True, blocksize=1024 * 1024)
%time df = dd.read_csv('../../data/raw/*.csv', encoding='iso-8859-1', dtype=data_types, assume_missing=True)

CPU times: user 2.75 s, sys: 219 ms, total: 2.97 s
Wall time: 2.98 s


In [6]:
# %time df.head()

In [7]:
%time df = df.fillna(-1)

CPU times: user 15.6 ms, sys: 0 ns, total: 15.6 ms
Wall time: 16.7 ms


In [8]:
# Takes a while
# %time df.count().compute()

In [9]:
# Takes a while, but should be doable
# %time unique_origins = df['Origin'].unique().compute()

In [10]:
# once you compute you get a real pandas series
# type(unique_origins)

In [11]:
# unique_origins

In [12]:
# 2400 is not a valid time
df['CRSDepTime'] = df.apply(lambda row: 2359 if row['CRSDepTime'] == 2400 else row['CRSDepTime'], axis='columns')

In [13]:
# df.apply?

In [14]:
# pd.Timestamp?

In [15]:
# head = df.head()

[########################################] | 100% Completed |  0.6s


In [16]:
def create_timestamp (row):
    return pd.Timestamp('%s-%s-%s;%04d'%(row['Year'], row['Month'], row['DayofMonth'], row['CRSDepTime']))

In [17]:
# type(head)

pandas.core.frame.DataFrame

In [18]:
# head

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2000,1,28,5,1647.0,1647,1906.0,1859.0,HP,154.0,...,15.0,11.0,0.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,2000,1,29,6,1648.0,1647,1939.0,1859.0,HP,154.0,...,5.0,47.0,0.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,2000,1,30,7,-1.0,1647,-1.0,1859.0,HP,154.0,...,0.0,0.0,1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,2000,1,31,1,1645.0,1647,1852.0,1859.0,HP,154.0,...,7.0,14.0,0.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,2000,1,1,6,842.0,846,1057.0,1101.0,HP,609.0,...,3.0,8.0,0.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [19]:
# create a sample for dask to figure out the data types
transformation_sample = head.apply(create_timestamp, axis='columns')

In [20]:
type(transformation_sample)

pandas.core.series.Series

In [21]:
transformation_sample[0]

Timestamp('2000-01-28 16:47:00')

In [22]:
# meta_information = {'@timestamp': pd.Timestamp}
meta_information = transformation_sample

df['@timestamp'] = df.apply(lambda row: pd.Timestamp('%s-%s-%s;%04d'%(row['Year'], row['Month'], row['DayofMonth'], row['CRSDepTime'])),
                            axis='columns',
                           meta=meta_information)

In [23]:
df.head()

[########################################] | 100% Completed |  3.1s


,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,@timestamp
0,2000,1,28,5,1647.0,1647,1906.0,1859.0,HP,154.0,...,11.0,0.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,2000-01-28 16:47:00
1,2000,1,29,6,1648.0,1647,1939.0,1859.0,HP,154.0,...,47.0,0.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,2000-01-29 16:47:00
2,2000,1,30,7,-1.0,1647,-1.0,1859.0,HP,154.0,...,0.0,1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,2000-01-30 16:47:00
3,2000,1,31,1,1645.0,1647,1852.0,1859.0,HP,154.0,...,14.0,0.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,2000-01-31 16:47:00
4,2000,1,1,6,842.0,846,1057.0,1101.0,HP,609.0,...,8.0,0.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,2000-01-01 08:46:00


In [24]:
df.npartitions

1623

In [25]:
from pyelasticsearch import ElasticSearch, bulk_chunks

In [26]:
ES_HOST = 'http://localhost:9200/'
INDEX_NAME = "expo2009"
DOC_TYPE = "flight"
es = ElasticSearch(ES_HOST)

In [27]:
# https://pyelasticsearch.readthedocs.io/en/latest/api/#pyelasticsearch.ElasticSearch.bulk
def documents(records):
    for flight in records:
        yield es.index_op(flight)
        
def chunk_import(records):        
    # bulk_chunks() breaks your documents into smaller requests for speed:
    for chunk in bulk_chunks(documents(records=records),
                             docs_per_chunk=500,
                             bytes_per_chunk=10000):
        # We specify a default index and doc type here so we don't
        # have to repeat them in every operation:
        es.bulk(chunk, doc_type=DOC_TYPE, index=INDEX_NAME)

In [28]:
begin_partition = 0
end_partition = df.npartitions

# begin_partition = 1
# end_partition = 2

In [29]:
for partition_nr in range(df.npartitions):
    if partition_nr > end_partition:
        break
    if partition_nr < begin_partition:
        continue
    print ("Importing partition %d"%(partition_nr))
    partition = df.get_partition(partition_nr)
    records = partition.compute().to_dict(orient='records')
    print ("Importing into ES")    
    chunk_import(records)
    cnt = es.count('*')['count']
    print ("Datasets in ES: %d"%(cnt))        

Importing partition 0
[########################################] | 100% Completed |  3.1s
Importing into ES
Datasets in ES: 10093
Importing partition 1
[########################################] | 100% Completed |  3.6s
Importing into ES
Datasets in ES: 20654
[                                        ] | 0% Completed |  0.0s
[########################################] | 100% Completed |  3.4s
Importing into ES
Datasets in ES: 31239
[                                        ] | 0% Completed |  0.0s
[########################################] | 100% Completed |  3.5s
Importing into ES
Datasets in ES: 41702
Importing partition 4
[########################################] | 100% Completed |  3.9s
Importing into ES
Datasets in ES: 52403
Importing partition 5
[########################################] | 100% Completed |  4.4s
Importing into ES


KeyboardInterrupt: 